In [1]:
data_var = '2024-01-16'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9761,2024-01-16,Finlândia Korisliiga,13:30,Kataja,Helsinki Seagulls,4.29,1.20,170.5,1.81,1.85,9.5,1.96,1.76,n9I11hrr,0.233100,0.833333,0.552486,0.540541,0.066434,140.662,26.971649,0.191748,1.8,1.643168,0.912871,144.21,1.770,0.427025,0.241257,-6.0,102.292,8.250186,0.080653,2.4,1.341641,0.559017,110.00,1.116,0.094763,0.084913,25.0,69,100,2.09,1.10,134.446,106.030,0.795978,0.015456,0.076033,-0.02,-0.004,-822.500000,0.000000,0.0,0.000000,-0.37,-0.074,-2.702703,0.0,0.0,0.0
9762,2024-01-16,Europa Liga Waba Feminina,14:00,Duga F,Orlovi F,1.97,1.76,148.5,1.80,1.87,-3.5,1.85,1.85,lCaiyBUA,0.507614,0.568182,0.555556,0.534759,0.075796,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,225.836,292.130942,1.293553,1.8,1.643168,0.912871,88.50,3.270,4.606539,1.408727,34.0,66,75,24.00,1.18,0.000,0.000,0.079621,0.026974,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.0,0.0,0.0
9763,2024-01-16,Finlândia Korisliiga,13:30,Karhu Basket,Kobrat,1.23,3.97,162.5,1.87,1.79,-10.5,2.02,1.71,MRAEbjS0,0.813008,0.251889,0.534759,0.558659,0.064897,194.098,166.412931,0.857366,1.8,1.643168,0.912871,99.68,2.208,1.920214,0.869662,62.0,274.594,230.952993,0.841071,1.2,1.643168,1.369306,117.86,3.428,2.989109,0.871969,-1.0,89,83,1.12,1.42,140.414,202.704,0.745182,0.030912,0.117535,0.19,0.038,6.052632,0.000000,0.0,0.000000,0.07,0.014,212.142857,0.0,0.0,0.0
9764,2024-01-16,Finlândia Korisliiga,13:30,BC Nokia,Lahti Basketball,1.07,7.75,184.5,1.87,1.87,-16.5,2.09,1.66,ryCAaWCf,0.934579,0.129032,0.534759,0.534759,0.063612,189.280,85.280855,0.450554,2.4,1.341641,0.559017,114.68,2.012,0.944653,0.469510,25.0,330.986,175.975513,0.531671,1.2,1.643168,1.369306,637.65,3.584,1.390550,0.387988,-42.0,94,109,1.22,5.85,163.472,529.542,1.071082,0.000000,0.162163,1.30,0.260,0.269231,0.000000,0.0,0.000000,0.85,0.170,39.705882,0.0,0.0,0.0
9765,2024-01-16,Europa Liga Waba Feminina,13:30,Sloga Pozega F,Partizan F,1.49,2.50,146.5,1.87,1.79,-5.5,1.99,1.73,fk0mxiF4,0.671141,0.400000,0.534759,0.558659,0.071141,709.612,1008.387241,1.421040,1.8,1.643168,0.912871,353.60,8.188,9.978523,1.218676,32.0,251.678,324.915619,1.290997,1.2,1.643168,1.369306,94.83,3.366,4.347560,1.291610,16.0,80,87,4.42,1.09,0.000,0.000,0.357984,0.030912,0.098843,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9819,2024-01-16,Eua Ncaa,22:30,Seton Hall,St. Johns,1.95,1.87,146.5,1.91,1.91,8.5,1.37,0.00,SAjhhwt1,0.512821,0.534759,0.523560,0.523560,0.047580,157.532,42.396086,0.269127,2.4,1.341641,0.559017,173.94,2.300,0.545573,0.237206,-3.0,149.314,56.336361,0.377301,2.4,1.341641,0.559017,209.95,1.966,0.901155,0.458370,32.0,78,65,2.23,3.23,195.414,214.522,0.029617,0.000000,1.414214,6.47,1.294,0.734158,0.000000,0.0,0.000000,-0.84,-0.168,-5.178571,0.0,0.0,0.0
9820,2024-01-16,Eua Ncaa,23:00,Arkansas,Texas A&M,2.28,1.67,145.5,1.91,1.91,10.5,1.33,0.00,fRNoCv00,0.438596,0.598802,0.523560,0.523560,0.037399,137.642,49.625371,0.360539,1.2,1.643168,1.369306,219.64,1.966,0.877172,0.446171,-38.0,146.618,73.658445,0.502383,1.2,

## Apostas do dia

### Back Home V3 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
9796,23:00,Eua Nba,Phoenix Suns,Sacramento Kings,1.57,Back Home
9804,21:00,Eua Ncaa,Cincinnati,TCU,1.67,Back Home
